In [1]:
import pandas as pd
df = pd.read_csv('jantojun2020.csv', dtype=object)
existing = pd.read_csv('airlines-corgis.csv')

In [2]:
df.head(5)

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,MKT_UNIQUE_CARRIER,MKT_CARRIER_FL_NUM,TAIL_NUM,ORIGIN,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2020,1,1,1,3,1/1/2020,WN,5888,N951WN,ONT,...,95.0,122.0,74.0,363,2,8.0,0.0,27.0,0.0,33.0
1,2020,1,1,1,3,1/1/2020,WN,6276,N467WN,ONT,...,90.0,92.0,71.0,363,2,NaN,NaN,NaN,NaN,NaN
2,2020,1,1,1,3,1/1/2020,WN,4598,N7885A,ONT,...,70.0,68.0,57.0,333,2,NaN,NaN,NaN,NaN,NaN
3,2020,1,1,1,3,1/1/2020,WN,4761,N551WN,ONT,...,75.0,75.0,63.0,333,2,NaN,NaN,NaN,NaN,NaN
4,2020,1,1,1,3,1/1/2020,WN,5162,N968WN,ONT,...,80.0,67.0,57.0,333,2,NaN,NaN,NaN,NaN,NaN


In [3]:
existing.head(5)
existing.iloc[0:5, 6:]

,Statistics.# of Delays.Carrier,Statistics.# of Delays.Late Aircraft,Statistics.# of Delays.National Aviation System,Statistics.# of Delays.Security,Statistics.# of Delays.Weather,Statistics.Carriers.Names,Statistics.Carriers.Total,Statistics.Flights.Cancelled,Statistics.Flights.Delayed,Statistics.Flights.Diverted,Statistics.Flights.On Time,Statistics.Flights.Total,Statistics.Minutes Delayed.Carrier,Statistics.Minutes Delayed.Late Aircraft,Statistics.Minutes Delayed.National Aviation System,Statistics.Minutes Delayed.Security,Statistics.Minutes Delayed.Total,Statistics.Minutes Delayed.Weather
0,1009,1275,3217,17,328,"American Airlines Inc.,JetBlue Airways,Contine...",11,216,5843,27,23974,30060,61606,68335,118831,518,268764,19474
1,374,495,685,3,66,"American Airlines Inc.,Alaska Airlines Inc.,Co...",14,138,1623,3,7875,9639,20319,28189,24400,99,77167,4160
2,296,477,389,8,78,"American Airlines Inc.,Continental Air Lines I...",11,29,1245,15,6998,8287,13635,26810,17556,278,64480,6201
3,300,472,735,2,54,"American Airlines Inc.,Continental Air Lines I...",11,73,1562,14,7021,8670,14763,23379,23804,127,65865,3792
4,283,268,487,4,58,"American Airlines Inc.,Alaska Airlines Inc.,Co...",13,74,1100,18,5321,6513,13775,13712,20999,120,52747,4141


In [4]:
#Start similar dataframe from new data
new = pd.DataFrame()
new['Airport.Code'] = df["ORIGIN"]

#Create dictionary of airport codes : airport names
codes = pd.Series(existing['Airport.Name'].values, existing['Airport.Code'].values).to_dict()

#Add airport names to the new dataframe
L = list(codes.keys()) + list(codes.values())
new['Airport.Name'] = new['Airport.Code'].str.extract('(' + '|'.join(L) + ')', expand=False).replace(codes)

In [8]:
#Add date information
new['Time.Label'] = df['YEAR'] + '/0' + df['MONTH']
new['Time.Month'] = df['MONTH']
months = {'1':'January', '2':'February', '3':'March', '4':'April', '5':'May', '6':'June'}
new['Time.Month Name'] = new['Time.Month'].replace(months)
new['Time.Year'] = df['YEAR']

#Create dataframe that will hold info about each individual flight
new_indiv = new.copy(deep=True)

#Ignore flights from airports not in original dataset
new = new.dropna(subset=['Airport.Name'])

#Reduce rows to one for each Airport.Code/Time.Label combo (like exisiting dataframe)
new = new.drop_duplicates(subset=['Airport.Code', 'Time.Label'])

new

,Airport.Code,Airport.Name,Time.Label,Time.Month,Time.Month Name,Time.Year
26,PDX,"Portland, OR: Portland International",2020/01,1,January,2020
59,PHL,"Philadelphia, PA: Philadelphia International",2020/01,1,January,2020
83,PHX,"Phoenix, AZ: Phoenix Sky Harbor International",2020/01,1,January,2020
358,SAN,"San Diego, CA: San Diego International",2020/01,1,January,2020
527,SEA,"Seattle, WA: Seattle/Tacoma International",2020/01,1,January,2020
...,...,...,...,...,...,...
2512520,BOS,"Boston, MA: Logan International",2020/06,6,June,2020
2521737,FLL,"Fort Lauderdale, FL: Fort Lauderdale-Hollywood...",2020/06,6,June,2020
2521750,JFK,"New York, NY: John F. Kennedy International",2020/06,6,June,2020
2521895,IAD,"Washington, DC: Washington Dulles International",2020/06,6,June,2020


In [7]:
test = new.dropna(subset=['Airport.Name'])
test.reset_index(inplace=True)
test = test.drop('index', axis=1)
test['Time.Month'], test['Time.Year'] = [df['MONTH'], df['YEAR']]
test

,Airport.Code,Airport.Name,Time.Label,Time.Month,Time.Month Name,Time.Year
0,PDX,"Portland, OR: Portland International",2020/01,1,January,2020
1,PHL,"Philadelphia, PA: Philadelphia International",2020/01,1,January,2020
2,PHX,"Phoenix, AZ: Phoenix Sky Harbor International",2020/01,1,January,2020
3,SAN,"San Diego, CA: San Diego International",2020/01,1,January,2020
4,SEA,"Seattle, WA: Seattle/Tacoma International",2020/01,1,January,2020
...,...,...,...,...,...,...
169,BOS,"Boston, MA: Logan International",2020/06,1,June,2020
170,FLL,"Fort Lauderdale, FL: Fort Lauderdale-Hollywood...",2020/06,1,June,2020
171,JFK,"New York, NY: John F. Kennedy International",2020/06,1,June,2020
172,IAD,"Washington, DC: Washington Dulles International",2020/06,1,June,2020
